In [12]:
import cv2
import numpy as np
import dlib
import time
import pyautogui

In [13]:
## 인식기 load 및 상수 파라미터 지정
# 얼굴 인식 model, 랜드마크 인식기 생성
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./content/shape_predictor_68_face_landmarks.dat')
pyautogui.FAILSAFE = False

# 양 눈 영역을 나타내는 랜드마크 인덱스
left_eye = [36, 37, 38, 39, 40, 41]
right_eye = [42, 43, 44, 45, 46, 47]
eyes = [left_eye , right_eye]

In [14]:
## 초기값 설정
threshold_bin = 80 # 이진화 임계값 설정 (: 클수록 화소값=0인 영역(검정색영역) 넓어짐)
threshold_dis = 25 # 좌표 차이 임계값 (: 작을수록 시선움직임에 민감하게 반응)
sensitivity = 50 # 민감도 (: 클수록 마우스의 움직임이 커진다)
output_width, output_height = 1920, 1080 # 내 모니터 출력창 크기
output_ratio = 1 
dx = dy = 0
point_d = ld = rd = left_pprev = right_pprev = left_p = right_p = (0,0)
# flags

In [16]:
def crop_and_binarization(img, rect, name):
    def resize_and_show(name, img):
    if img is not None:
        _h, _w = img.shape[:2]
        cv2.namedWindow(name, cv2.WINDOW_NORMAL)  
        cv2.resizeWindow(name, _w*output_ratio, _h*output_ratio)    
        cv2.imshow(name, img)
        
    # 이미지 자르기
    x1, y1 = rect[0]
    x2, y2 = rect[2]
    cropped_img = img[y1:y2, x1:x2]

    # 자른 이미지를 이진화
    #binary_image = get_binary_img(cropped_img)
    gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    blur = cv2.GaussianBlur(equalized, (5, 5), 0)
    _, binary_image = cv2.threshold(blur, threshold_bin, 255, cv2.THRESH_BINARY)

    # 객체 영역과 나머지 영역을 분할하여 결과 이미지 생성
    binarized_img = np.where(binary_image <= threshold_bin, 0, 255).astype(np.uint8)
    
    # resize and show
    resize_and_show(name, binarized_img)

    return binarized_img


In [17]:
def find_centroid(binary_image):
    # 검은색 영역의 좌표 찾기
    black_pixels = np.where(binary_image == 0)
    x_coords = black_pixels[1]
    y_coords = black_pixels[0]

    # 무게 중심을 계산하기 위해 좌표의 합과 픽셀 개수 계산
    sum_x = np.sum(x_coords)
    sum_y = np.sum(y_coords)
    num_pixels = len(x_coords)

    # 무게 중심 계산
    centroid_x = sum_x / num_pixels
    centroid_y = sum_y / num_pixels

    #draw_coordinate(frame, coordinate, position)

    return centroid_x, centroid_y

In [18]:
def init_point():
    pyautogui.moveTo(output_width//2, output_height//2)
    time.sleep(1)

In [19]:
def get_eye_pts(landmarks, eye):
    # 눈 영역(사각형)의 x, y 좌표를 저장할 리스트 초기화
    eye_pts = []
    # 눈 영역을 crop할 사각 바운더리 계산
    mid_side_h = np.mean([landmarks.part(eye[0]).y, landmarks.part(eye[3]).y])
    up_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[1]).y, landmarks.part(eye[2]).y]))
    up_side = mid_side_h - up_side_dd
    low_side_dd = 2 * abs(mid_side_h - np.mean([landmarks.part(eye[4]).y, landmarks.part(eye[5]).y]))
    low_side = mid_side_h + low_side_dd
    left_side = landmarks.part(eye[0]).x 
    right_side = landmarks.part(eye[3]).x 

    eye_pts.append((left_side, up_side))
    eye_pts.append((right_side, up_side)) 
    eye_pts.append((right_side, low_side)) 
    eye_pts.append((left_side, low_side))     

    eye_pts_np = np.array(eye_pts, np.int32)

    return eye_pts_np

In [20]:
def draw_coordinate(frame, coordinate, position):
    if coordinate is not None:
        text = f"({int(coordinate[0])}, {int(coordinate[1])})"
    else:
        text = "None"
    
    if position == "left":
        text_position = (10, frame.shape[0] - 10)
    elif position == "right":
        text_position = (frame.shape[1] - 120, frame.shape[0] - 10)
    elif position == "middle":
        text_position = (frame.shape[1] // 2 - 100, frame.shape[0] - 10)
    else:
        raise ValueError("Invalid position. Available positions are 'bottom_left' and 'bottom_right'.")
    
    cv2.putText(frame, text, text_position,
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

In [ ]:
point_prev = None
flag_dc = False
flag_lc = False
flag_rc = False
threshold_dis = 25
stop_mode_start_time = None

def process_frame(point):
    global point_prev, flag_dc, flag_lc, flag_rc, stop_mode_start_time
    
    # 좌표 추출
    if point is not None:
        x, y = point
        if point_prev is not None:
            dx = x - point_prev[0]
            dy = y - point_prev[1]
            
            # 이동 거리 계산
            distance_squared = dx**2 + dy**2
            
            # 정지 모드 체크
            if distance_squared < threshold_dis:
                if stop_mode_start_time is None:
                    stop_mode_start_time = time.time()
            else:
                stop_mode_start_time = None
        
            # 마우스 조작
            if stop_mode_start_time is not None and time.time() - stop_mode_start_time > 3:
                if flag_dc:
                    pyautogui.doubleClick(x, y)
                    flag_dc = False
                elif flag_lc:
                    pyautogui.click(x, y)
                    flag_lc = False
                elif flag_rc:
                    pyautogui.rightClick(x, y)
                    flag_rc = False
        else:
            dx = dy = None
        
        # 마우스 이동
        if not flag_dc and not flag_lc and not flag_rc and dx is not None and dy is not None:
            pyautogui.move(dx, dy)
        
        point_prev = (x, y)
    
    # 프레임 처리 완료 후 초기화
    if point is None:
        point_prev = None
        flag_dc = False
        flag_lc = False
        flag_rc = False
        stop_mode_start_time = None


In [1]:
## main()
# ##실행 시 반드시 얼굴이 있어야 오류가 뜨지 않음##
global point_prev, flag_dc, flag_lc, flag_rc, stop_mode_start_time
init_trig = True
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Failed to open webcam")

while True:
    # 프레임 읽어오기
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    # 영점보정 트리거=="i" 시 -> 마우스를 화면의 정중앙으로 이동(1초)
    # 안먹힐때 많음 << 왜인지 모름..
    if cv2.waitKey(1) & 0xFF == ord('i'):
        init_trig = True
    if init_trig:
        init_point()
        init_trig = False

    # 그레이스케일 후 얼굴 검출
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if not faces:
        continue

    for face in faces:
        # 얼굴의 68개 landmarks 검출 
        landmarks = predictor(gray, face)
        left_eye_pts_np= get_eye_pts(landmarks, eyes[0])
        right_eye_pts_np= get_eye_pts(landmarks, eyes[1])

    # L, R 안구 이미지 자르고 이진화
    left_eye_biimg = crop_and_binarization(frame, left_eye_pts_np, "L_eye")
    right_eye_biimg = crop_and_binarization(frame, right_eye_pts_np, "R_eye")
    #resize_and_show("L_eye", left_eye_biimg)
    #resize_and_show("R_eye", right_eye_biimg)
    # 시선 좌표 탐지
    left_center = find_centroid(left_eye_biimg)
    right_center = find_centroid(right_eye_biimg)
    draw_coordinate(frame, left_center, "left")
    draw_coordinate(frame, right_center, "right")

    # 두 쪽 모두 검출된 경우: 3초 이상 '정지모드'일 때 더블클릭
    if left_center is not None and right_center is not None:
        flag_rc = False
        flag_dc = True
        flag_lc = False
        average_x = (left_center[0] + right_center[0]) / 2
        average_y = (left_center[1] + right_center[1]) / 2
        point = (average_x, average_y)
    # 왼쪽만 검출된 경우: "" 일 때 우클릭
    elif left_center is not None:
        flag_rc = True
        flag_dc = False
        flag_lc = False
        point = left_center
    # 오른쪽만 검출된 경우: "" 일 때 좌클릭
    elif right_center is not None:
        flag_rc = False
        flag_dc = False
        flag_lc = True
        point = right_center
    # 아무것도 검출되지 않았을 경우: stay doing nothing & 초기화
    else:
        flag_rc = False
        flag_dc = False
        flag_lc = False
        point = None
    
    process_frame(point)
            
    # 화면에 프레임 출력
    cv2.imshow("Result", frame)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠 해제 및 창 닫기
cap.release()
cv2.destroyAllWindows()

IndentationError: expected an indented block after 'if' statement on line 49 (3813848950.py, line 54)